#YOLO

In [ ]:
yolo_dataset_path = "./datasets/"
yolo_weights_path = "./best_try12.pt"

# Collego Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp "/content/drive/MyDrive/Colab Notebooks/ProgettoML4VM/Datasets/GTSDB_Yolo_trafficlight.zip" ./

In [ ]:
!cp "/content/drive/MyDrive/Colab Notebooks/ProgettoML4VM/Yolo/weights/yoloGTSDB/best_try12.pt" ./

In [ ]:
!unzip ./GTSDB_Yolo_trafficlight.zip

# Installazione librerie necessarie

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO
import cv2
from google.colab.patches import cv2_imshow
import tqdm
import os
import numpy as np

 Link guide

 https://github.com/ultralytics/ultralytics

 https://docs.ultralytics.com/usage/python/#predict

# Funzioni di servizio

In [ ]:

# Intersection over union
def IoU(y_true, y_pred):

  ix1 = np.maximum(y_true[0], y_pred[0])
  iy1 = np.maximum(y_true[1], y_pred[1])
  ix2 = np.minimum(y_true[2], y_pred[2])
  iy2 = np.minimum(y_true[3], y_pred[3])

  intersection_height = np.maximum(iy2 - iy1 + 1, 0.0)
  intersection_width = np.maximum(ix2 - ix1 + 1, 0.0)

  intersection_area = intersection_height * intersection_width

  height_true = y_true[3] - y_true[1] + 1
  width_true = y_true[2] - y_true[0] + 1

  height_pred = y_pred[3] - y_pred[1] + 1
  width_pred = y_pred[2] - y_pred[0] + 1

  # va sottratta intersection area per non contarla 2 volte
  union_area = height_true*width_true + height_pred*width_pred - intersection_area

  IoU = intersection_area / union_area

  return IoU

In [ ]:
def readCorrectLabel(file_path, w_img, h_img):

  prediction = []

  with open(file_path, 'r') as f:

    file_content = f.read()
    for line in file_content.split("\n"):
      if(len(line)==0):
        continue
      (cls, x, y, w, h) = line.split(" ")
      x, y, w, h = float(x), float(y), float(w), float(h)
      x, y, w, h = int(x*w_img), int(y*h_img), int(w*w_img), int(h*h_img)
      x1, y1, x2, y2 = int(x-w/2), int(y-h/2), int(x+w/2), int(y+h/2)
      cls = int(cls)
      label = (cls, x1, y1, x2, y2)
      prediction.append(label)


    return prediction

# Classi

In [ ]:
classNames = ["SpeedLimit 20 km/h", "SpeedLimit 30 km/h", "SpeedLimit 50 km/h", "SpeedLimit 60 km/h", "SpeedLimit 70 km/h", "SpeedLimit 80 km/h", "RestrictionEnds 80 km/h", "SpeedLimit 100 km/h", "SpeedLimit 120 km/h", "NoOvertaking", "NoOvertaking (trucks) ", "PriorityAtNextIntersection (danger)", "PriorityRoad", "GiveWay", "Stop", "NoTrafficBothWays", "NoTrucks", "NoEntry", "Danger", "BendLeft (danger)", "BendRight (danger)", "Bend (danger)", "UnevenRoad (danger)", "SlipperyRoad (danger)", "RoadNarrows (danger)", "Construction (danger)", "GeneralDanger", "PedestrianCrossing (danger)", "SchoolCrossing (danger)", "CyclesCrossing (danger)", "Snow (danger)", "Animals (danger)", "RestrictionEnds", "GoRight (mandatory)", "GoLeft (mandatory)", "GoStraight (mandatory)", "GoRightOrStraight (mandatory)", "GoLeftOrStraight (mandatory)", "KeepRight (mandatory)", "KeepLeft (mandatory)", "Roundabout (mandatory)", "RestrictionEnds (overtaking)", "RestrictionEnds (overtaking trucks)", "green trafficlight", "red trafficlight", "yellow trafficlight"]

classNamesITA = ["limite velocità 20kmh", "limite velocità 30kmh", "limite velocità 50kmh", "limite velocità 60kmh", "limite velocità 70kmh", "limite velocità 80kmh", "Fine restrizione 80kmh", "limite velocità 100kmh", "limite velocità 120kmh", "sorpasso vietato", "sorpasso vietato (Camion)", "dare precedenza al prossimo incrocio", "strada con priorità", "dare precedenza", "stop", "accesso vietato", "vietato (camion)", "accesso vietato", "pericolo generico", "curva sinistra", "curva destra", "curva", "strada sconnessa", "strada scivolosa", "restringimento carreggiata", "lavori in corso", "pericolo", "attraversamento pedonale", "attraversamento scolari", "attraversamento bici", "neve", "animali", "fine restrizione", "obbligo svoltare destra", "obbligo svoltare sinistra", "obbligo proseguire dritti", "procedere dritto o destra", "procedere dritto o a sinistra", "tenere la destra", "tenere la sinistra", "rotatoria", "fine restrizione sorpasso", "fine restrizione sorpasso (camion)", "semaforo verde", "semaforo rosso", "semaforo giallo"]


print(f"\n Il dataset contiene {len(classNames)} classi\n")

for i  in range(len(classNames)):
  print(f" class {i:02d}:   {classNames[i]:40s} {classNamesITA[i]:40s}")



# Carico il modello

In [ ]:
myModel = YOLO(yolo_weights_path)

In [ ]:
myModel.val(
    data="./data_ita.yaml",
    split="test",
    iou=0.5, # for non max suppression
    )

# Valuto i risultati sul test set

In [ ]:
# Statistiche
number_of_detected_object = 0
IoU_sum = 0.0
number_of_object = 0
false_detection_tot = 0

threshold = 0.5
confidence = 0.6


# Questa è la predizione faccio la predizione di tutte le immagini nella cartella del validation
results = myModel(yolo_dataset_path+"/images/test", stream=True, conf=confidence)

# stream=True fa la predizione solo quando si va a leggere results cosi da non saturare la ram


for result in results:

  print(f"\n{result.path}:")

  res_plotted = result.plot()
  cv2_imshow(res_plotted)

  # Stampo le predizioni
  print("\nPredicted:")
  boxes = result.boxes
  for i in range( len(boxes) ):
    cls = int(boxes.cls[i])
    conf = float(boxes.conf[i])
    x1, y1, x2, y2 = boxes.xyxy[i]
    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
    cv2.rectangle(res_plotted, (x1, y1), (x2, y2), (0, 255, 0), 2)
    print(f" - {classNames[cls].upper():40s} (class number: {cls}) (confidence: {100*conf:.1f}%)")

  # Confronto con l'etichetta del dataset
  print("\nActual:")
  path = result.path.replace(".jpeg", ".txt")
  path = path.replace("images", "labels")
  height, width, channels = res_plotted.shape
  labels = readCorrectLabel(path, width, height)

  predicted_labels = boxes.xyxy

  for label in sorted(labels):
    number_of_object+=1;
    c, x1, y1, x2, y2 = label
    c = int(c)
    detected = False
    iou = 0.0
    # Ricerco il bounding box corretto nelle predizioni
    for i, predicted_label in enumerate(predicted_labels):
      if c==boxes.cls[i]:
        detected=True
        x1, y1, x2, y2 = predicted_label
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        tmp = IoU(label[1:5], (x1, y1, x2, y2))
        if tmp>iou:
          iou=tmp
    if detected and iou>threshold:
      print(f" - {classNames[label[0]].upper():40s} (class number: {label[0]}) IoU: {100*iou:.2f}%")
      number_of_detected_object+=1
      IoU_sum+=iou
    else:
      print(f" - {classNames[label[0]].upper():40s} (class number: {label[0]}) Not Detected")

  # Rilevo i False detection
  false_detection = 0
  n=len(predicted_labels)
  found = [False for i in range(n)]
  for i, predicted_label in enumerate(predicted_labels):
    x1, y1, x2, y2 = predicted_label
    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
    for label in labels:
        c, x1, y1, x2, y2 = label
        c = int(c)
        if c==boxes.cls[i] and found[i]==False and IoU(label[1:5], (x1, y1, x2, y2))>threshold:
          found[i]=True
  for i in range(n):
    if found[i] == False:
      false_detection+=1
  print(f"\nFalse Detection: {false_detection}")
  false_detection_tot+=false_detection


  print('='*100)

# Statistiche

In [ ]:
print(f"""
        \n - Oggetti rilevati: {number_of_detected_object}/{number_of_object} ({100*number_of_detected_object/number_of_object:.2f}%)
        \n - IoU Medio: {100*IoU_sum/number_of_detected_object:.2f}%
        \n - False Detection: {false_detection_tot:d}
      """)